In [22]:
import requests
from bs4 import BeautifulSoup
import numpy as np

In [2]:
url = "https://www.nyc.gov/site/tlc/about/tlc-trip-record-data.page"
response = requests.get(url)
if response.status_code == 200:
    print("Success")
else:
    print("Failure")

Success


In [4]:
import re
results_page = BeautifulSoup(response.content,'lxml')

#### get data from January 2009 through June 2015

In [5]:
def get_taxi_link():
    link_array = []
    pattern = r"(http.*yellow_tripdata_(20[0|1][5-9])-(0?\d*).parquet)"
    # pattern = "https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2015-01.parquet"
    match = re.findall(pattern, str(results_page))
    for i in match:
        if int(i[1]) < 2019:
            link_array.append(i[0])
        elif int(i[2]) <= 6:
            link_array.append(i[0])
    return link_array

In [6]:
link = get_taxi_link()

In [27]:
pattern = r"yellow.*.parquet"
match = re.search(pattern, link[0])
match.group()

'yellow_tripdata_2019-01.parquet'

In [ ]:
pattern = r"yellow.*.parquet"
for taxi_data_url in link:
    match = re.search(pattern, taxi_data_url)
    name_of_file = match.group()
    response = requests.get(taxi_data_url, stream=True)
    with open(name_of_file, 'wb') as f:
        for chunk in response.iter_content(chunk_size=1024):
            if chunk:
                f.write(chunk)
                

In [7]:
link[0]

'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2019-01.parquet'

In [9]:
import pandas as pd
from pyarrow.parquet import ParquetFile
df = pd.read_parquet(link[0])

In [19]:
index = df.columns

In [24]:
np.array(index)

array(['VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime',
       'passenger_count', 'trip_distance', 'RatecodeID',
       'store_and_fwd_flag', 'PULocationID', 'DOLocationID',
       'payment_type', 'fare_amount', 'extra', 'mta_tax', 'tip_amount',
       'tolls_amount', 'improvement_surcharge', 'total_amount',
       'congestion_surcharge', 'airport_fee'], dtype=object)

In [11]:
import geopandas as gpd
shapefile = gpd.read_file("taxi_zones.shp")
shapefile

,OBJECTID,Shape_Leng,Shape_Area,zone,LocationID,borough,geometry
0,1,0.116357,0.000782,Newark Airport,1,EWR,"POLYGON ((933100.918 192536.086, 933091.011 19..."
1,2,0.433470,0.004866,Jamaica Bay,2,Queens,"MULTIPOLYGON (((1033269.244 172126.008, 103343..."
2,3,0.084341,0.000314,Allerton/Pelham Gardens,3,Bronx,"POLYGON ((1026308.770 256767.698, 1026495.593 ..."
3,4,0.043567,0.000112,Alphabet City,4,Manhattan,"POLYGON ((992073.467 203714.076, 992068.667 20..."
4,5,0.092146,0.000498,Arden Heights,5,Staten Island,"POLYGON ((935843.310 144283.336, 936046.565 14..."
...,...,...,...,...,...,...,...
258,259,0.126750,0.000395,Woodlawn/Wakefield,259,Bronx,"POLYGON ((1025414.782 270986.139, 1025138.624 ..."
259,260,0.133514,0.000422,Woodside,260,Queens,"POLYGON ((1011466.966 216463.005, 1011545.889 ..."
260,261,0.027120,0.000034,World Trade Center,261,Manhattan,"POLYGON ((980555.204 196138.486, 980570.792 19..."
261,262,0.049064,0.000122,Yorkville East,262,Manhattan,"MULTIPOLYGON (((999804.795 224498.527, 999824...."


According to the shapefile "taxi_zones.shp" above, the highest objectID, which is the pickup location ID and drop off location ID, is 263. Therefore, in the Cleaning & filtering step, we remove the IDs that are greater than 263.

In [12]:
nc_counties = shapefile['geometry']
nc_counties = nc_counties.to_crs(4326)
nc_counties['lon'] = nc_counties.centroid.x  
nc_counties['lat'] = nc_counties.centroid.y

/var/folders/sl/ygvmf08n6s5cz73c1jv6rbnw0000gn/T/ipykernel_54596/953358834.py:3: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  nc_counties['lon'] = nc_counties.centroid.x


TypeError: One of the arguments is of incorrect type. Please provide only Geometry objects.

In [14]:
nc_counties.y

TypeError: One of the arguments is of incorrect type. Please provide only Geometry objects.

In [15]:
nc_counties['lon']

AttributeError: 'numpy.ndarray' object has no attribute 'crs'